In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath('../'))
from sklearn.model_selection import train_test_split

from TumorDetection.data.loader import DataPathLoader
from TumorDetection.data.dataset import TorchDataset, Dataset
from TumorDetection.utils.dict_classes import DataPathDir, Device, ReportingPathDir
from TumorDetection.models.efsnet import EFSNet
from TumorDetection.models.utils.lightning_model import LightningModel
from TumorDetection.models.utils.trainer import Trainer

## TODO